# VCF Updating rs ID
Problem: Compare data from VCF and TXT files, with the VCF file being the main file. If the columns 1 (chromosome), 2 (position), and Alternative sequence columns of both files match, insert the rs_ID data from the existing variant column of the TXT file as the third column of the VCF file.

## 1. Explore Main VCF

In [7]:
from fuc import pyvcf
import pandas as pd
import numpy as np

In [8]:
#Construct VcfFrame from a VCF file
vcf = pyvcf.VcfFrame.from_file('SP01_vep.vcf')

In [110]:
#check cvf
vcf.df.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,10,94426,.,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
1,10,95429,.,C,CA,. . .,.,CSQ=A|upstream_gene_variant|MODIFIER|TUBB8|ENS...
2,10,120756,.,A,T,. . .,.,CSQ=T|regulatory_region_variant|MODIFIER|||Reg...
3,10,126237,.,G,A,. . .,.,CSQ=A|downstream_gene_variant|MODIFIER|IL9RP2|...
4,10,126402,.,T,C,. . .,.,CSQ=C|non_coding_transcript_exon_variant|MODIF...


In [10]:
print(vcf.df.dtypes)

CHROM     object
POS        int64
ID        object
REF       object
ALT       object
QUAL      object
FILTER    object
INFO      object
dtype: object


## 2. Explore Information File (TXT) for Updating

In [11]:
txt_df = pd.read_csv('SP01_vep.txt', sep='\t', header=0)

In [5]:
txt_df.head()

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,...,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,PHENOTYPES,NMD,CADD_PHRED,CADD_RAW,Mastermind_MMID3,GO
0,.,10:94426-94426,T,intron_variant,MODIFIER,TUBB8,ENSG00000173876,Transcript,ENST00000309812.4,protein_coding,...,-,-,-,-,Oocyte_maturation_defect_2+ClinVar+rs10904045,-,4.434,0.110377,-,"GO:0003924:GTPase_activity,GO:0005200:structur..."
1,.,10:94426-94426,T,intron_variant,MODIFIER,TUBB8,ENSG00000173876,Transcript,ENST00000332708.5,protein_coding,...,-,-,-,-,Oocyte_maturation_defect_2+ClinVar+rs10904045,-,4.434,0.110377,-,-
2,.,10:94426-94426,T,downstream_gene_variant,MODIFIER,RP11-631M21.6,ENSG00000237297,Transcript,ENST00000416477.1,unprocessed_pseudogene,...,-,-,-,-,Oocyte_maturation_defect_2+ClinVar+rs10904045,-,4.434,0.110377,-,-
3,.,10:94426-94426,T,intron_variant,MODIFIER,TUBB8,ENSG00000173876,Transcript,ENST00000447903.2,protein_coding,...,-,-,-,-,Oocyte_maturation_defect_2+ClinVar+rs10904045,-,4.434,0.110377,-,"GO:0003924:GTPase_activity,GO:0005200:structur..."
4,.,10:95429-95429,A,upstream_gene_variant,MODIFIER,TUBB8,ENSG00000173876,Transcript,ENST00000309812.4,protein_coding,...,-,-,-,-,-,-,2.885,0.016923,-,"GO:0003924:GTPase_activity,GO:0005200:structur..."


In [12]:
#checks all column
txt_df.columns

Index(['#Uploaded_variation', 'Location', 'Allele', 'Consequence', 'IMPACT',
       'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON',
       'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position',
       'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation',
       'REF_ALLELE', 'UPLOADED_ALLELE', 'DISTANCE', 'STRAND', 'FLAGS',
       'SYMBOL_SOURCE', 'HGNC_ID', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL',
       'APPRIS', 'CCDS', 'SIFT', 'PolyPhen', 'AF', 'AFR_AF', 'AMR_AF',
       'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF',
       'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF',
       'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF',
       'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF',
       'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF',
       'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO',
       'PUBMED', 'MOTIF_NAME', 'M

In [14]:
#Colume Allele here is ALT
txt_df = txt_df[['Location', 'Allele', 'REF_ALLELE', 'Existing_variation']]
txt_df.head()

,Location,Allele,REF_ALLELE,Existing_variation
0,10:94426-94426,T,C,"rs10904045,COSV59118495"
1,10:94426-94426,T,C,"rs10904045,COSV59118495"
2,10:94426-94426,T,C,"rs10904045,COSV59118495"
3,10:94426-94426,T,C,"rs10904045,COSV59118495"
4,10:95429-95429,A,-,rs35442274


In [15]:
print(txt_df.dtypes)

Location              object
Allele                object
REF_ALLELE            object
Existing_variation    object
dtype: object


In [15]:
#the location need to be separated
from rsUpdate_vcf import split_location

#test
split_location('10:94426-94426')

('10', '94426')

In [10]:
#Split 'Location' txt using : into 2 new columns 'CHROM', 'POS'
txt_df = txt_df.assign(
    CHROM=txt_df['Location'].apply(lambda x: split_location(x)[0]),
    POS=txt_df['Location'].apply(lambda x: split_location(x)[1]),
)

In [11]:
txt_df

,Location,Allele,REF_ALLELE,Existing_variation,CHROM,POS
0,10:94426-94426,T,C,"rs10904045,COSV59118495",10,94426
1,10:94426-94426,T,C,"rs10904045,COSV59118495",10,94426
2,10:94426-94426,T,C,"rs10904045,COSV59118495",10,94426
3,10:94426-94426,T,C,"rs10904045,COSV59118495",10,94426
4,10:95429-95429,A,-,rs35442274,10,95429
...,...,...,...,...,...,...
1084226,X:155238653-155238653,T,A,rs2472749,X,155238653
1084227,X:155238653-155238653,T,A,rs2472749,X,155238653
1084228,X:155238653-155238653,T,A,rs2472749,X,155238653
1084229,X:155238653-155238653,T,A,rs2472749,X,155238653


In [16]:
#the rs ID need to be extracted from txt
from rsUpdate_vcf import extract_id

#test
extract_id('rs10904045,COSV59118495')

'rs10904045'

In [22]:
txt_df['ID'] = txt_df['Existing_variation'].apply(extract_id)
txt_df['ID']

0          rs10904045
1          rs10904045
2          rs10904045
3          rs10904045
4          rs35442274
              ...    
1084226     rs2472749
1084227     rs2472749
1084228     rs2472749
1084229     rs2472749
1084230     rs2472749
Name: ID, Length: 1084231, dtype: object

## 3. Updating main VCF file

**Step 1:** Read and construct vcfFrame using pyvcf from the original VCF file.

**Step 2:** Read the TXT file and extract only the desired columns: 'Location', 'Allele', 'REF_ALLELE', and 'Existing_variation'.

Note:
- Ensure 'Existing_variation' is of type str before using the extract_id function.
- Ensure the 'POS' data in the TXT file is of type int, as it matches the type in the VCF file.

**Step 3:** Merge the two DataFrames on the columns CHROM, POS, and ALT/Allele. Select relevant columns from the VCF DataFrame and fill empty cells with '.'.

**Step 4:** Retrieve the header from the original VCF file.

**Step 5:** Create the modified VCF file.

In [1]:
from rsUpdate_vcf import main

files_to_update = ['SP01', 'SP04', 'SP09', 'SP13', 'SP24', 'SP31']
for file in files_to_update:
    main(file)

/Users/chiranan/Desktop/Python/GitHub/VCF_Test/rsUpdate_vcf.py:49: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  txt_df = pd.read_csv(f'{sp_num}_vep.txt', sep='\t', header=0)
/Users/chiranan/Desktop/Python/GitHub/VCF_Test/rsUpdate_vcf.py:49: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  txt_df = pd.read_csv(f'{sp_num}_vep.txt', sep='\t', header=0)


In [6]:
#example of original SP13
vcf_SP13_original = pyvcf.VcfFrame.from_file('SP13_vep.vcf')
vcf_SP13_original.df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,10,94545,.,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
1,10,95032,.,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
2,10,95226,.,A,T,. . .,.,CSQ=T|5_prime_UTR_variant|MODIFIER|TUBB8|ENSG0...
3,10,95229,.,C,T,. . .,.,CSQ=T|5_prime_UTR_variant|MODIFIER|TUBB8|ENSG0...
4,10,95429,.,C,CA,. . .,.,CSQ=A|upstream_gene_variant|MODIFIER|TUBB8|ENS...
...,...,...,...,...,...,...,...,...
434927,7,77958262,.,T,A,. . .,.,CSQ=A|intron_variant|MODIFIER|MAGI2|ENSG000001...
434928,7,77958559,.,T,C,. . .,.,CSQ=C|intron_variant|MODIFIER|MAGI2|ENSG000001...
434929,7,77961831,.,T,C,. . .,.,CSQ=C|intron_variant|MODIFIER|MAGI2|ENSG000001...
434930,7,77967743,.,G,T,. . .,.,CSQ=T|intron_variant|MODIFIER|MAGI2|ENSG000001...


In [5]:
#example of updated SP13
vcf_SP13_update = pyvcf.VcfFrame.from_file('SP13_updated.vcf')
vcf_SP13_update.df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,10,94545,rs10904047,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
1,10,95032,rs181491095,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
2,10,95226,rs6560829,A,T,. . .,.,CSQ=T|5_prime_UTR_variant|MODIFIER|TUBB8|ENSG0...
3,10,95229,rs6560830,C,T,. . .,.,CSQ=T|5_prime_UTR_variant|MODIFIER|TUBB8|ENSG0...
4,10,95429,.,C,CA,. . .,.,CSQ=A|upstream_gene_variant|MODIFIER|TUBB8|ENS...
...,...,...,...,...,...,...,...,...
434927,7,77958262,.,T,A,. . .,.,CSQ=A|intron_variant|MODIFIER|MAGI2|ENSG000001...
434928,7,77958559,.,T,C,. . .,.,CSQ=C|intron_variant|MODIFIER|MAGI2|ENSG000001...
434929,7,77961831,.,T,C,. . .,.,CSQ=C|intron_variant|MODIFIER|MAGI2|ENSG000001...
434930,7,77967743,.,G,T,. . .,.,CSQ=T|intron_variant|MODIFIER|MAGI2|ENSG000001...
